# Exporting from MOCHA for IGV Visualization

In this vignette, we will start from a MOCHA object and export data to a folder `./data/` for visualization in the accompanying Dash app.

1. [Imports](#Imports)
2. [Export Sample-Specific Coverage](#Export-Sample-Specific-Coverage)
3. [Export Open Tiles](#Export-Open-Tiles)
4. [Export Differential Tiles](#Export-Differential-Tiles)
5. [Annotate and Export Motifs](#Annotate-and-Export-Motifs)

## Imports

Install and import MOCHA, and load your output SampleTileObject from of `MOCHA::getSampleTileMatrix`.

In [1]:
devtools::install_github("aifimmunology/MOCHA", ref="exportBigWigs")
library(MOCHA)

Alternatively, you can load in the output from `MOCHA::callOpenTiles` and compute the SampleTileObject.

In [2]:
tileResults <- readRDS("../data/celltype_mocha_tileresults.rds")
SampleTileMatrices <- MOCHA::getSampleTileMatrix(
  tileResults, 
  cellPopulations = "All", 
  groupColumn = NULL, 
  threshold = 0, # union
  numCores = 8
)

## Export Sample-Specific Coverage

In [3]:
# Ensure all these options are set, as some are non-default
MOCHA::exportCoverage(
    SampleTileObj = tileResults,
    dir = "./data/sample_specific_coverage/",
    cellPopulations = "All",
    sampleSpecific = TRUE,
    saveFile = TRUE,
    numCores = 8
)

Loading required package: RaggedExperiment

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



ERROR: Error in MOCHA::exportCoverage(SampleTileObj = tileResults, dir = "./data/sample_specific_coverage/", : Directory given by SampleTileObj@metadata$Directory does not exist.


## Export Open Tiles

In [ ]:
MOCHA::exportOpenTiles(
    SampleTileObject = SampleTileMatrices,
    outDir = "./data/tiles_samplespecific/"
)

## Export Differential Tiles

Load a GRanges of differential tiles, or calculate differentials with `MOCHA::getDifferentialAccessibletiles`.

In [ ]:
DifferentialsGRList <- readRDS("./data/teaseq_drug-vs-dmso-diff-peaks.rds")

In [ ]:
exportDifferentialTiles(
    SampleTileObject = SampleTileMatrices, 
    DifferentialsGRList, 
    outDir = "./data/tiles_differential/"
)

## Annotate and Export Motifs

Annotate our peaks with a motif set PWM. There is a selection of PWMs available in the package [chomVarMotifs](https://github.com/GreenleafLab/chromVARmotifs).

In [ ]:
library(chromVARmotifs)
data("human_pwms_v1")

motifsGRanges <- MOCHA::addMotifSet(
    SampleTileObj = SampleTileMatrices, 
    motifPWMs = human_pwms_v1,
    returnSTM = FALSE, # Set FALSE to return a GRanges
    motifSetName = "CISBP"
)

We have the ability to export motifs that are annotated within all tiles, or filter to only motifs located within open tiles for each cell population with the option `filterCellTypePeaks`.

In [ ]:
exportMotifs(
    SampleTileObject = tileResults,
    motifsGRanges = unlist(motifsGRanges),
    motifSetName = "CISBP",
    outDir = "./data/motifs/", 
    filterCellTypePeaks = TRUE,
    verbose = TRUE
)